In [1]:
import qutip
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import pickle
import time as timess
import math, cmath
import sys
import auxiliary_library as me

from IPython.display import display, Math, Latex

In [3]:
N = 6
spin_ops_list = me.one_body_spin_ops(N)

### Heisenberg Hamiltonian
chain_type = "XXZ"; Hamiltonian_paras = [.2, .15, .1, 1.]
H_H = me.Heisenberg_Hamiltonian(spin_ops_list, chain_type, N, False, Hamiltonian_paras, False)

### 

magnetization = sum(spin_ops_list[3][a] for a in range(len(spin_ops_list[3])))
neel_operator = sum((-1)**a * spin_ops_list[3][a] for a in range(len(spin_ops_list[3])))

### n_oc_op = sum((spin_ops_list[3][a] + .5 * qutip))
cl_ops, label_ops = me.classical_ops(H_H, N, spin_ops_list, False)
cl_ops["Neel_op"] = neel_operator; label_ops.append("Neel_op")
me.hamiltonian_comm_check(H_H, cl_ops)

psi0 = ((1-.1j)*qutip.tensor([qutip.basis(2,1) for k in range(int(N/2)-1)]+
                    [qutip.basis(2,0)]+
                    [qutip.basis(2,1) for k in range(int(N/2+.5))])
        -
        (1+.1j)*qutip.tensor([qutip.basis(2,1) for k in range(int(N/2))]+
                    [qutip.basis(2,0)]+
                    [qutip.basis(2,1) for k in range(int(N/2+.5)-1)]))*np.sqrt(.5/1.01)

[H,  x_op ] = 0?:  False
[H,  p_op ] = 0?:  False
[H,  n_oc_op ] = 0?:  False
[H,  comm_xp ] = 0?:  False
[H,  corr_xp ] = 0?:  False
[H,  p_dot ] = 0?:  False
[H,  Neel_op ] = 0?:  False
